In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
from pandas.io import sql
import MySQLdb
from sqlalchemy import create_engine

## Import Raw Data

> Import raw data from csv to a dataframe

In [2]:
class Extractor:
    
    def __init__(self, path):
        
        self.path = path
    
    def load_csv(self, path):
               
        df = pd.read_csv(path)
        
        return df

In [3]:
path = r'C:\Users\benja\OneDrive\Documents\python\Traffic_Data\20181024_dX_0830_0900.csv'

df = Extractor(path)
df = df.load_csv(path)
df.head(5)

,track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time
0,1; Car; 10.18; 36.634649; 37.984642; 23.724906...
1,2; Car; 21.81; 39.253912; 37.984630; 23.725042...
2,3; Motorcycle; 29.68; 38.165999; 37.984594; 23...
3,4; Car; 40.72; 36.645291; 37.984582; 23.725250...
4,5; Car; 6.81; 12.251624; 37.984506; 23.725572;...


## Restructure Data
> Since the data we got is in a format that isn't very useful to us, we need to restructure our dataframe. We want our data in a format where each cell is a data point, except for trajectory which will have several data points 

In [4]:
class Restruct:
    
    def __init__(self, data):
        
        self.data = data
   
    def restructure(self, data):
        track_ids = []
        types = []
        traveled_d = []
        avg_speeds =[]
        trajectories = []
    
        for r in range(len(data)):
            row = data.iloc[r,:][0].split(';')  # seperates values at every semicolon
            row_p1 = row[:4]                    # takes all of the values but trajectory
            row_p2 = row[4:]                    # takes all of the trajectory values
            trajectory = ','.join(row_p2)       # combines trajectory values
        
            track_ids.append(row_p1[0])          # populates track_ids list
            types.append(row_p1[1])              # populates types list
            traveled_d.append(row_p1[2])         # populates traveled_d list
            avg_speeds.append(row_p1[3])         # populates avg_speeds list
            trajectories.append(trajectory[1:])  # populates trajectory list
        
        columns = data.columns[0].split(';')[:4] # seperates column labels into individual values
        columns.append('trajectory')             # adds trajectory label to dataframe
        columns[1] = 'types'                     # changes name of column label to types
        for i in range(len(columns)):
            columns[i] = columns[i].strip()      # creates new column labels and removes empty space in data
            
    
        # creates dictionary with new labels and seperated data lists 
        data_dict = {columns[0]:track_ids, columns[1]:types, columns[2]:traveled_d, columns[3]:avg_speeds, columns[4]:trajectories}
        new_df = pd.DataFrame(data_dict) # creates dataframe from dictionary
    
        print('dataframe successfully created')
        return new_df

In [5]:
new_df = Restruct(df)
new_df = new_df.restructure(df)

new_df.head()

dataframe successfully created


,track_id,types,traveled_d,avg_speed,trajectory
0,1,Car,10.18,36.634649,"37.984642, 23.724906, 38.1611, 0.0000, 0.0000,..."
1,2,Car,21.81,39.253912,"37.984630, 23.725042, 40.0550, 0.0000, -0.0000..."
2,3,Motorcycle,29.68,38.165999,"37.984594, 23.725116, 38.3354, -0.0245, -0.283..."
3,4,Car,40.72,36.645291,"37.984582, 23.725250, 37.3981, -0.0227, 0.0415..."
4,5,Car,6.81,12.251624,"37.984506, 23.725572, 10.9666, -0.0423, -0.802..."


## Warehouse Data

> Now we connect to sql in order to create databases and tables to house our data, and populate those tables with the data from the dataframes we created

In [11]:
class Database_Build:

    def __init__(self, host, username, password, table_name, database_name, df):
        
        
        self.host = host
        self.username = username
        self.password = password
        self.table_name = table_name        
        self.database_name = database_name
        self.df = df
        
    def connect_to_server(self, host, username, password):
        
        try:
            cnct = mysql.connector.connect(host = host, username = username, password = password) # creates a connection to sql server
            print("MySQL Server Connection Successful")
        except Error as err:
            print(f"Error: '{err}'")
        
        return cnct
    
    def create_db(self, database_name):
        
        try:
            mycursor = cnct.cursor()                              # allows us to write sql queries
            mycursor.execute(f"CREATE DATABASE {database_name}")  # executes query to create data of the name that we specified
            print("Database Created Successfully")
        except Error as err:
            print(f"Error: '{err}'")
            
        return mycursor
    
    def connect_to_db(self, host, username, password, database_name):
        
        try:
            mydb = mysql.connector.connect(host = host, username = username, password = password, database = database_name)  # connects to specified sql db
            print("MySQL Database Connection Successful")
        except Error as err:
            print(f"Error: '{err}'")
        
        return mydb
    
    def make_table(self, df, table_name):
        
        column = list(df.columns.values)
        
        try:
            table = f"CREATE TABLE IF NOT EXISTS {table_name}("        
            for col in range(len(column)):
                table += '\n' + column[col] + ' ' + input(f"Enter data type for {column[col]}: ") + ','  
            table = table[:-1] + ')'
        
            mycursor = mybase.cursor()
            mycursor.execute(table)
            print("Table Created Successfully or Already Exists")
        except Error as err:
            print(f"Error: '{err}'")
        
        return table
    
    def pop_table(self, df, table_name, host, username, password, database_name):
        
        engine = create_engine(f'mysql://{username}:{password}@{host}/{database_name}') # creates a connection to the sql db
            
        fill_table = df.to_sql(con = engine, name = table_name, if_exists = 'replace')
        
        return fill_table

In [7]:
host = 'localhost'
username = 'root'
password = 'password'
database_name = 'Trafficdata'
table_name = 'traffic_data'
df = new_df




cnct = Database_Build(df, host, username, password, database_name, table_name)
cnct = cnct.connect_to_server(host,username,password)

MySQL Server Connection Successful


In [8]:
mycursor = Database_Build(df, host, username, password, database_name, table_name)
mycursor = mycursor.create_db(database_name)

Error: '1007 (HY000): Can't create database 'trafficdata'; database exists'


In [9]:
mybase = Database_Build(df, host, username, password, database_name, table_name)
mybase = mybase.connect_to_db(host, username, password, database_name)

MySQL Database Connection Successful


In [12]:
table = Database_Build(df, host, username, password, database_name, table_name)
table = table.make_table(df, table_name)

Enter data type for track_id: INT
Enter data type for types: VARCHAR(40)
Enter data type for traveled_d: FLOAT
Enter data type for avg_speed: FLOAT
Enter data type for trajectory: FLOAT
Table Created Successfully or Already Exists


In [13]:
dataf = new_df

fill_table = Database_Build(df, host, username, password, database_name, table_name)
fill_table = fill_table.pop_table(dataf, table_name, host, username, password, database_name)